In [1]:
import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer
from sklearn.linear_model import LogisticRegression
from bs4 import BeautifulSoup

In [2]:
wnl=WordNetLemmatizer()

In [3]:
stopwords=set(w.rstrip() for w in open('D:\\Tutorials\\Data Science Natural Language Processing (NLP) in Python\\NLP Data\\Sentiment Analyzer\\stopwords.txt'))

In [4]:
pos_rev=BeautifulSoup(open('D:\\Tutorials\\Data Science Natural Language Processing (NLP) in Python\\NLP Data\\Sentiment Analyzer\\positive.reviews').read())
pos_rev=pos_rev.findAll('review_text')

In [5]:
neg_rev=BeautifulSoup(open('D:\\Tutorials\\Data Science Natural Language Processing (NLP) in Python\\NLP Data\\Sentiment Analyzer\\negative.reviews').read())
neg_rev=neg_rev.findAll('review_text')

In [15]:
np.random.shuffle(pos_rev)
pos_rev=pos_rev[:len(neg_rev)]

In [16]:
def my_tokenizer(s):
    s=s.lower()
    tokens=nltk.tokenize.word_tokenize(s)
    tokens=[t for t in tokens if len(t)>2]
    tokens=[wnl.lemmatize(t) for t in tokens]
    tokens=[t for t in tokens if t not in stopwords]
    return tokens

In [17]:
pos_tokenized=[]
neg_tokenized=[]

In [18]:
word_index_map={}
current_index=0
for review in pos_rev:
    tokens=my_tokenizer(review.text)
    pos_tokenized.append(tokens)
    for token in tokens:
        if token not in word_index_map:
            word_index_map[token]=current_index
            current_index+=1

In [19]:
for review in neg_rev:
    tokens=my_tokenizer(review.text)
    neg_tokenized.append(tokens)
    for token in tokens:
        if token not in word_index_map:
            word_index_map[token]=current_index
            current_index+=1

In [23]:
def tokens_to_vector(tokens,label):
    x=np.zeros(len(word_index_map)+1)
    for t in tokens:
        i=word_index_map[t]
        x[i]+=1
    x=x/x.sum()
    x[-1]=label
    return x
N=len(pos_tokenized)+len(neg_tokenized)
data=np.zeros((N,len(word_index_map)+1))
i=0
for tokens in pos_tokenized:
    xy=tokens_to_vector(tokens,1)
    data[i,:]=xy
    i+=1
for tokens in neg_tokenized:
    xy=tokens_to_vector(tokens,0)
    data[i,:]=xy
    i+=1

In [24]:
np.random.shuffle(data)
X=data[:,:-1]
y=data[:,-1]

In [25]:
X_train=X[:-100,]
y_train=y[:-100,]
X_test=X[-100:,]
y_test=y[-100:,]
model=LogisticRegression()
model.fit(X_train,y_train)
print('Train set acc:',model.score(X_train,y_train))
print('Test set acc:',model.score(X_test,y_test))

Train set acc: 0.7079843662758235
Test set acc: 0.68


In [27]:
threshold=0.5
for word,index in word_index_map.items():
    weight=model.coef_[0][index]
    if weight>threshold or weight<-threshold:
        print(word,weight)

card -0.6059247616840455
ha 0.6201122831377617
month -0.7066564775485176
space 0.5738421442401013
you 0.9448811249848311
time -0.5528530123246125
using 0.687270086066657
've 0.5161556416217755
love 1.1345288723195721
lot 0.659021868238785
wa -1.664913422808285
sound 1.2115581865877574
little 0.9246519974592425
n't -1.917322707035652
buy -0.9134376950215942
price 2.6489657886751767
customer -0.5952165712274425
look 0.5616482822422956
money -1.0198165920275395
size 0.546193499507105
video 0.5620662374853169
excellent 1.2873163121953284
happy 0.6263504225271604
home 0.5621951373697451
unit -0.6732298080951128
recommend 0.6201207763720699
speaker 1.1095124743431277
perfect 0.7746946324235422
cable 0.6135062217789219
picture 0.5689809923212316
fast 1.0334637650158303
doe -1.1489795460869987
quality 1.4736773634893392
easy 1.5555742534347712
tried -0.7106899323146009
paper 0.522033537197292
highly 1.0009492961834245
value 0.5266516707267199
memory 0.81148223317687
company -0.5068076671070832